In [1]:
import numpy as np
import math
from scipy import optimize
import numba as nb
import cap_equi as func
import interactive_plots as int_plot

import scipy.stats as stats

# For plots:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
plt.style.use('seaborn-whitegrid')
mpl.style.use('seaborn')
prop_cycle = plt.rcParams["axes.prop_cycle"]
colors = prop_cycle.by_key()["color"]
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# PS2, part 1: Welfare effects of capital injection

The exercise consists of the core setup:
* Two periods, $t=0,1$,
* Two types of agents: Consumers and entrepreneurs. 
* **Consumers:**
    * Linear utility $u_c = c_0^c+c_1^c$.
    * Large endowment of consumption goods in each period: $e$. 
    * Labor endowment of $e_l=1$. Inelastically supplied in a competitive labor market at the price $w_1$.
* **Entrepreneurs:**
    * Linear utility $u_e = c_0^e+c_1^e$.
    * Small endowment $n_0$ in period 0. 
    * Borrow $b_1$, invest in capital $k_1$. 
    * Period 1 they hire labor $l_1$ at wage $w_1$.
    * Entrepreneurs utilize capital and labor in $t=1$ with Cobb-Douglas production function 
    $$ \begin{align} 
    y_1 = k_1^{\alpha}l_1^{1-\alpha} \notag
    \end{align} $$
    * **Entrepreneurs are borrowing constrained:**
    $$\begin{align} 
    b_1\leq \lambda \left(y_1-w_1l_1\right), && \lambda\in (0,1) \notag
    \end{align} $$
* Assume that endowment of consumption goods $(e)$ is *sufficiently* large, for the interest rate to be 1 in equilibrium. (We will show what *sufficiently* means here).

## Q1: Three parts. (i) State the entrepreneur's problem. (ii) Argue that entrepreneurs will choose to maximize profits in period 1, a linear function of $k_1$, and (3) restate entrepreneur's problem as a simple linear problem.

### (i) Entrepreneur problem:

Go through the steps:

1. For the entrepreneur write up the maximization problem. This inludes:
    * Objective function, what is maximized? Constraints on the problem are
    * The budget for $t=0$, (with "=")
    * the budget for $t=1$, (with "=")
    * the production function, (with "=")
    * the credit constraint (with "$\leq$").
    Let us refer ot this as **(Problem 1)**.
    
2. Next, substitute for budgets $t=0,t=1$ into the utility function.
3. Substitute for the production function in both the function from step 2., and the credit constraint. We end up with what we refer to as **problem 2**, that should look like something like this:

$$\begin{align}
    \max_{k_1,l_1,b_1} n_0+k_1^{\alpha}l_1^{1-\alpha}-k_1-w_1l_1, && s.t. b_1\leq \lambda \left(k_1^{\alpha}l_1^{1-\alpha}-w_1l_1\right). \tag{Problem 2}
\end{align}$$ 




### (ii) Optimal choice of labor:

Note that profits in $t=1$ is given by:

$$\begin{align}
    \pi_1 = k_1^{\alpha}l_1^{1-\alpha}-w_1l_1.
\end{align}$$

Find the first order conditions in **(Problem 2)** defined above, and show that this implies:

$$\begin{align}
    w_1 = (1-\alpha)k_1^{\alpha}l_1^{-\alpha}
\end{align}$$

and thus also 

$$\begin{align} 
w_1l_1^* = (1-\alpha)k_1^{\alpha}(l_1^*)^{1-\alpha}, \tag{Labor Share}
\end{align}$$ 

and 

$$\begin{align}
    l_1^* = \dfrac{(1-\alpha)^{1/\alpha}k_1}{w_1^{1/\alpha}}. \tag{$l^*$}
\end{align}$$

Use this to write the **profit function** as:

$$\begin{align}
    \pi_1(l=l^*) &= \underbrace{\dfrac{\alpha (1-\alpha)^{(1-\alpha)/\alpha}}{w_1^{(1-\alpha)/\alpha}}}_{\equiv R(w_1)}k_1 \tag{Profit}
\end{align}$$







### (iii) Restate as linear problem. 

Use the results above. Refer to this linear problem as **(Problem 3)**.

## Q2: Derive entrepreneur's demand for capital, and argue how the solution depends on $\lambda$ and $R(w_1)$.

Problem $3$ is linear in $k_1$. Thus capital demand depends on the range of the interest rate. Explain the three potential solutions:

* If $R(w_1)<1$?
* If $\lambda R(w_1)>1$?
* If $\lambda R(w_1)<1<R(w_1)$?
* If $R(w_1)=1$?

In the figures below you can see how the demand changes with parameter values of e.g. $\lambda$. (The other line in the graph is $(\alpha k^{\alpha-1})$ which also has to hold in equilibrium) 


.

.

.

.

.

.

.

Hint: Capital demand in the different ranges are given by (not necessarily correct order compared to bullets above)

$$\begin{align} 
     k_1^* &= \dfrac{n_0}{1-\lambda R(w_1)} \\ 
     k_1^* &= 0 \\ 
     k_1^* &= \infty \\
      k_1^*&\in\left[0,\mbox{ }\dfrac{n_0}{1-\lambda}\right] \\ 
\end{align}$$


In [2]:
# Baseline parameters:
n_0base, lambda_base, alpha_base, Inf = 0.1, 0.55, 0.9, 10**3

# Exogenous grid of R (interest rate):
R_min, R_max, N_R= 0.1, 2, 100
R_grid = np.linspace(R_min, R_max, N_R)

# Upper and lower limit of sliders to operate below:
n_0_lower, n_0_upper, n_0_step = 0, 0.5, 0.05
lambda_lower, lambda_upper, lambda_step = 0, 0.9, 0.05

# Plot:
int_plot.interactive_cap(n_0base, lambda_base, alpha_base, Inf, N_R, R_grid, n_0_lower, n_0_upper, n_0_step, lambda_lower, lambda_upper, lambda_step)

interactive(children=(FloatSlider(value=0.55, description='$\\lambda$', max=0.9, step=0.05), FloatSlider(value…

In [3]:
# Grids:
n_0_lower, n_0_upper, N_n = 0, 0.5, 51  
lambda_lower, lambda_upper, N_lambda = 0, 0.9, 91
n_grid = np.linspace(n_0_lower, n_0_upper,N_n, dtype=np.float)
lambda_grid, lambda_step = np.linspace(lambda_lower, lambda_upper, N_lambda, retstep=True, dtype=np.float)
# baseline:
n_0base, lambda_base, alpha = 0.1, 0.5, 0.9
# Plot:
int_plot.interactive_equicap(N_n,N_lambda,alpha,n_grid,lambda_grid,math.ceil(N_lambda/2),lambda_lower, lambda_upper,lambda_step)

interactive(children=(FloatSlider(value=0.46, description='$\\lambda$', max=0.9, step=0.01), Output()), _dom_c…

## Q3: Show the cutoff $n^*$, where $n_0 \geq n^*$$ implies that the first-best level of capital can be financed, and the borrowing constraint is not binding.

You can think of the following:

* Impose $l_1=1$ in equilibrium (endowment of labor from consumers, supplied inelastically).
* Define the first-best level of capital, as the one that **maximizes net ouptut** $(y_1-k_1)$.
* In equilibrium we further use that the market for capital is competitive, implying that the rate of return on capital has to follow:

$$\begin{align}
    R(w_1) = \alpha k_1^{\alpha-1}, && \Rightarrow && k_1 = \left(\dfrac{\alpha}{R(w_1)}\right)^{1/(1-\alpha)}
\end{align}$$

Given this and the capital demand function from Q2, when is the first-best level feasible?


## Q4: Show that if n<n* there is an equilibrium where the entrepreneurs are constrained and $k_1$ becomes an increasing function of $n_0$.

It follows from Q2-Q3 that the threshold exists, and that the entrepreneur is credit-constrained in this region. Now:

* Use the relevant capital demand from Q2. 
* Plug in the expression for equilibrium $R(w_1)$. 
* Solve for $k$ in this expression.

**Note:** When the entrepreneur is credit-constrained, $k_1$ is not only an increasing function of $n_0$, the effect of increased equity is **more** than 1-to-1. Why is this the case?

## Q5: Introduce a lump-sum tax on consumers in period 0 that transfers wealth to entrepreneurs. Derive expected utility as a function of the lump sum tax rate.

Hints:

* Split up the consideration into the region where E is credit-constrained, and where E is not.
* You can use all of the solutions from above and simply replace $n_0$ with $n_0+\tau$ for the entrepreneur.

You do not need to arrive at exactly this expression, but show that the effect of an increase in $\tau$ has the following effect on utility of entrepreneurs:

$$\begin{align}
    \dfrac{\partial U_e}{\partial \tau} = \left\lbrace \begin{array}{ll}  1 & n_0+\tau\geq n^* \\
    \dfrac{\alpha R(w_1)(1-\lambda)}{1-\alpha \lambda R(w_1)} & n_0+\tau<n^* \end{array}\right.
\end{align}$$

## Q6: Show that the relationship between consumer utility and the lump-sum tax is non-monotone. Show that for sufficiently small $n_0$ the tax can be pareto-improving.

Entrepreneurs always benefit from the tax. 

For consumers the tax includes three potential effects. Show that the effect on utility is given by:

$$\begin{align}
    \dfrac{\partial U_c}{\partial \tau} = \left\lbrace \begin{array}{ll} \color{red}{-1}, & n_0+\tau\geq n^* \\ \color{red}{-1} +\color{blue}{(1-\alpha)R(w_1)}\color{blueviolet}{\dfrac{\partial k_1}{\partial \tau}}, & n_0+\tau<n^*\end{array}\right.
\end{align}$$

Explain the three effects (here marked with different colors).

## Q7: Discuss how this justifies government's capital injection during the financial crisis. What features might the model be missing?

## Q8: How does key findings in Q6 depend on the type of borrowing constraint?

# PS2, part 2: Costly state verification and optimality of debt

The core of the model setup:

* **Utility** is linear: $U_E=c_0+c_1$.
* Entrepreneur invests at time $0$, cost $I$. Return at time $t=1$ is random, distributed according to $g(s)$:

$$\begin{align}
    Return = Rs, && s\sim g(s), && \text{Support on }S\equiv \left[s^{min}, s^{max}\right]
\end{align}$$
Also assumed that
$$\begin{align}
    \mathbb{E}[s]=1, && R>I.
\end{align}$$

* **Asymmetric information:** State $s$ is unknown at time $t=0$. At time $t=1$ the entrepreneur (E) observes the true value of $s$ costlessly. The financier (F) can choose to audit and observe it, but at the cost $\kappa>0$.
* **Assumption:** The financier chooses a *deterministic* audit rule. I.e. no *random/stochastic* audit rule. By the *revelation principle:* Only focus on contracts in which E has no incentive to lie. 
* **Contracts:** Specified by the triplet

$$\begin{align}
    Contract = \left( R^E(s), \mbox{ }R^F(s), \mbox{ }S_A\right), && S_A\subset S
\end{align}$$
where $R^E(s)$ and $R^F(s)$ denotes the payoffs for E,F (net of audit cost). $S_A$ is the set of states $S$, where $F$ chooses to audit.

* **Resource constraint:**

$$\begin{align}
    R^E(s)+R^F(s)+\kappa \mathbf{1}_{s\in S_A} = Rs, && R^E(s)\geq 0.
\end{align}$$

where $\mathbf{1}_{s\in S_A}$ indicates auditing, and the inequality captures *limited liability* of the entrepreneur.

* **Participation constraint:** (for F)

$$\begin{align}
    \int_S R^F(s)g(s)\mbox{ }ds\geq I.
\end{align}$$

States that expected revenue covers investment cost.

### Q1: Write down the IC constraint for E. What does it imply for F's net payoff?

A few notes before you try this:

* **IC constraint** with a continuous set of possible outcomes: *In any given state $s$ it must not be profitable to deviate to *any* state $s'$.*
* A useful theroem/principle: **The revelation principle**. The principle roughly states that every potential equilibrium outcome, can be achieved by an incentive-compatible strategy.

**What does it imply for F's net payoff?**

* What does the (IC) imply for two states $s,s'$ that are both in the no-audit region? (Remember the IC constraint has to hold for both states)
* Now consider the case where $s\in S_A$ (alternative s' is not in audit region): What does the (IC) state in this case?
* Does F receive higher/lower revenue in the audit region than in no-audit region?

### Q2: Define a standard debt contract:

$$\begin{align}
    R^F(s) = \left\lbrace \begin{array}{ll} R\bar{s}, & s\geq \bar{s} \\ Rs-\kappa, & s\leq \bar{s} \end{array}\right., && S_A=\left[s^{min},\mbox{ }\bar{s}\right].
\end{align}$$

### Check that any level $\bar{s}$ satisfies both the resource constraint and the (IC).

### Q3: Prove that the standard contract is optimal.

This is a bit tricky. Read the hint given in the problem text, it provides some guidance.

### Q4: Assume that $s$ is uniform. Characterizes the riskiness of the optimal debt contract.

Assume that $s$ is uniform on $[1-\Delta, 1+\Delta]$. In other words the pdf (f) and cdf (F) are given by:

$$\begin{align}
f(s) = \dfrac{1}{2\Delta}, && F(s) = \dfrac{s-(1-\Delta)}{2\Delta}, && s\in [1-\Delta, 1+\Delta]
\end{align}$$

Now set up the following:

* The entrepreneur's exptected profits when (1) $s\leq \bar{s}$ the entrepreneur receives 0 and when $s\geq \bar{s}$ he receives $Rs-R\bar{s}$.
* E maximizes expected profits **given** the financier participates, i.e. has non-negative expected profits. Explain that it looks like:

$$\begin{align}
    \dfrac{1}{2\Delta} \int_{\bar{s}}^{1+\Delta} R\bar{s}\mbox{ }ds+\dfrac{1}{2\Delta}\int_{1-\Delta}^{\bar{s}}(Rs-\kappa)\mbox{ }ds\geq I
\end{align}$$

(note that you can easily integrate this out, as it is only linear in $s$).

**Characterizing E's optimal contract:**

To characterize the optimal debt contract, consider E's motive for choosing $\bar{s}$: If he did not have to consider F's constraint, what is the optimal level then?

Hint 1: The figure below illustrates how the expected profits (participation constraint, PC) varies with the level of investment costs, $I$ and the audit costs.
.

.

.

.

.

(Hint: If you can, try to show that the optimal $s^*$ for E is given by:)

$$\begin{align}
 \bar{s}^* = \left\lbrace \begin{array}{ll}1+\Delta-\dfrac{\kappa}{R}-\dfrac{1}{R}\sqrt{(R(1+\Delta)-\kappa)^2+2R*\left[(1-\Delta)\kappa-\dfrac{R}{2}(1-\Delta)^2-2\Delta I\right]}, & R(1-\Delta)<I \\ 1-\Delta, & R(1-\Delta)\geq I \end{array}\right.
\end{align}$$

In [4]:
# Parameter values:
R = 2
I = 1.5
Delta = 0.5
kappa = 0.1
# Grid of sbar:
sbar = np.linspace(1-Delta, 1+Delta, 101)
# PC constraint:
int_plot.interactive_pc(R,I,Delta,kappa,sbar)
s_star = 1+Delta-kappa/R-(1/R)*math.sqrt((R*(1+Delta)-kappa)**2+2*R*((1-Delta)*kappa-(R/2)*(1-Delta)**2-2*Delta*I))
print(s_star)

interactive(children=(FloatSlider(value=1.5, description='I', max=2.0), FloatSlider(value=0.1, description='$\…

0.8155711229775239


### Q5: Find the threshold cost $I$, where the project is profitable. Comment on the solution.

Use the result in Q4: The choice of $s^*$ follows one of two scenarios:

* (Scenario 1) If $R(1-\Delta)\geq I$, what is the level of $s^*$? What is the maximum level of $I$ where the participation constraint (PC) holds?
* (Scenario 2) If $R(1-\Delta)<I$, how is $s^*$ determined? (Hint PC holds w. equality)

Hint 1: To show the maximum level of $I$ in scenario 2, note that the (PC) equation with equality is a *quadratic equation*. This has a solution as long as the determinant $\geq 0$.

.

.

.

.

. 

Hint 2: The maximum investment level is given by:

$$\begin{align}
     I^{max} = \max \Bigg\lbrace \underbrace{R-\kappa+\dfrac{\kappa^2}{4\Delta R}}_{\text{Scenario 1}},\mbox{ }\mbox{ }\underbrace{R(1-\Delta)}_{\text{Scenario 2}}\Bigg\rbrace.
\end{align}$$
